# [CptS 215 Data Analytics Systems and Algorithms](https://github.com/gsprint23/cpts215)
[Washington State University](https://wsu.edu)

[Gina Sprint](http://eecs.wsu.edu/~gsprint/)
# B-trees

Learner objectives for this lesson:
* Introduce B-trees and B+ trees
* Discuss applications of B-trees and B+ trees


## Acknowledgments
Content used in this lesson is based upon information in the following sources:
* [Dr. Ananth Kalyanaraman](http://www.eecs.wsu.edu/~ananth/)'s CptS 223 notes

## Non-binary Trees
We have spent a good amount of time discussing binary trees, binary search trees, and AVL trees. Now we are going to turn our attention to other tree implementations that are non-binary. For example, a 2-3 tree consists of nodes that are either "2-nodes" or "3-nodes". A 2-node is the same as a BST node, that is the node has two children and the data in the left subtree is smaller than the data at the current node and the data in the right subtree is greater than the data at the current node. On the other hand, a 3-node contains two data fields (x, y) such that the first data field (x) is less than the second data field (y). A 3-node, unlike a 2-node, has three children. The data in the left subtree is smaller than the data field x, the data in the middle subtree is greater than x but less than y, and the data in the right subtree is greater than the data field y. 

Here is an example 2-3 tree:
<img src="http://cs.middlesex.cc.nj.us/~schatz/csc236/worksheets/2-3tree.q2.final.png" width="500">
(image from [http://cs.middlesex.cc.nj.us/~schatz/csc236/worksheets/2-3tree.q2.final.png](http://cs.middlesex.cc.nj.us/~schatz/csc236/worksheets/2-3tree.q2.final.png))

In the above example the 2-nodes and the 3-nodes are as follows:
* 2-nodes: 50, 20, 10, 5, 15, 35, 90, 60, 70, 80, 100, 95
* 3-nodes: 25,30; 40,45; 65,75; 105,110

Note: in order to remain balanced, a 2-3 tree requires that all leaf nodes are at the lowest level.

The 2-3 tree is a specific case of a more general non-binary tree, the B-tree. A B-tree allows up to n children per node and was designed for building indexes into quite large databases stored on disk. We are going to focus the rest of this lesson on B-trees because in data analytics you are going to be working with big data, that is large databases and datasets that are too big to fit in RAM. It is important to understand how B-trees are used to manage such large amounts of data.

## B-tree Motivation
Let's take a look at how large databases can be! 
<img src="https://raw.githubusercontent.com/gsprint23/cpts215/master/lessons/figures/largest_databases.png" width="500">

And this data is 10+ years old! You can imagine how big the databases of today may be. This is where B-trees come in to help manage such large databases. A B-tree is a tree data structure for disks. Let's talk about how disks (secondary storage) compare to primary storage:

||Primary storage|Secondary storage|
|-|-|-|
|Hardware|RAM (main memory), cache|Disk (i.e. I/O)|
|Storage capacity|> 100 MB to 2-4GB|Giga ($10^9$) to Terabytes ($10^{12}$) to ...|
|Data persistence|Transient (erased after process terminates)|Persistent (permanently stored)|
|Data access speeds|~ a few clock cycles (i.e. x $10^{-9}$ seconds)|milliseconds ($10^{-3}$) = data seek time + data read time (could be a million times slower than main memory read)|

What if we used a balanced BST to store store information from a large database like Google (33 trillion items). The estimated access time (if we use a simple balanced BST) would be: $h = \mathcal{O}(log_{2}33 \times 10^{12}) \approx 44.9$ disk accesses. If we assume 120 disk accesses per second, then each search takes approximately 0.37 seconds!! Let's also take into account that 1 disk access is about $10^6$ CPU instructions. What happens if you need to do a million searches??

What we really need to do in order to help speed up searches in large trees is to reduce the height of a tree. BST and AVL trees at best have heights $log_{2}(n)$. If $n=10^6$, then $log_{2}(10^{6}) \approx 20$. Twenty disk seeks for each level would be too much! So we need to reduce the height by increasing the log base beyond 2. For example, suppose we allowed for 5-ary trees, $log_{5}10^{6} < 9$. In general, instead of binary (2-ary) trees, we will use $M$-ary trees such that $M > 2$. 

Let's take a look at an example. Suppose we have a 3-way search tree. In a balanced 3-way search tree, the height is $log_{3}(n)$. Each node stores:
* <= 2 keys
* <= 3 references to its children
<img src="https://raw.githubusercontent.com/gsprint23/cpts215/master/lessons/figures/example_3way_tree.png" width="400">

<img src="https://raw.githubusercontent.com/gsprint23/cpts215/master/lessons/figures/mway_tree_levels.png" width="700">

## B-trees
In a general B-tree, we use a $M$-way search tree (tree height is $log_{M}(N)$). Each node access brings in $M - 1$ keys and $M$ child references. We want to choose $M$ so the node size is equivalent to 1 disk block size.

<img src="https://raw.githubusercontent.com/gsprint23/cpts215/master/lessons/figures/btree_node_structure.png" width="600">

<img src="https://raw.githubusercontent.com/gsprint23/cpts215/master/lessons/figures/using_btrees.png" width="700">

<img src="https://raw.githubusercontent.com/gsprint23/cpts215/master/lessons/figures/five_way_tree.png" width="700">

When designing our B-trees, we should be aware of the following factors:
<img src="https://raw.githubusercontent.com/gsprint23/cpts215/master/lessons/figures/btree_factors.png" width="700">

Now, let's take a look at an example. A standard disk block size is 8192 bytes. Let's assume keys (that uniquely identify data elements) use 32 bytes and child references use 4 bytes. This means the required space per node is $32 \times (M - 1) + 4 \times M = 8192$. Solving for $M$, we find out we can construct our trees to be 228-way, and $log_{228}(33\times10^{12}) = 5.7$ disk accesses. Each search will take 0.047 seconds.

## B+ Trees
A B+ tree of order $M$ is a $M$-way tree with all of the following properties (note the parameters $N$, $M$, and $L$):
1. Leaves store the real data items
1. Internal nodes store up to $M - 1$ keys, such that key $i$ is the smallest key in subtree $i + 1$
1. Root can have between 2 to $M$ children
1. Each internal node (except the root) has between $ceil(M/2)$ to $M$ children
1. All leaves are at the same depth
1. Each leaf has between $ceil(L/2)$ and $L$ data items for some $L$

For example, here is a B+ tree of order 5:
<img src="https://raw.githubusercontent.com/gsprint23/cpts215/master/lessons/figures/five_way_bplustree.png" width="800">

<img src="https://raw.githubusercontent.com/gsprint23/cpts215/master/lessons/figures/five_way_bplustree2.png" width="800">



### Designing a B+ Tree
How are we going to determine the number of children per node ($M$=?)? And how are we going to determine the number of data items per leaf node ($L$=?)? For the root and internal nodes, there are $M$ child references ($4 \times M$ bytes) and there are $M - 1$ key entries ($(M - 1) \times K$ bytes) where $K$ denotes the size of a key. For leaf nodes, there are at most $L$ number of data items per leaf, which means the storage needed per leaf is $L \times D$ where $D$ denotes the size of each data item ($K \leq D$). Values for $M$ and $L$ are chosen based on:
* Disk block size $B$
* Data element size $D$
* Key size $K$

The root node and each internal node needs $m = 4 \times M + (M - 1) \times K$ bytes. Each internal node has to fit inside a disk block, implying $B = 4M + (M - 1)K$. Solving the above yields $m = floor(\frac{B + K}{4 + K})$. For example, if $K=4$ KB and $B=8$ KB, then $m = 1024$ bytes is the threshold for internal node capacity.

For leaf nodes, $L = floor(\frac{B}{D})$. For example, if $D=4$ KB and $B=8$ KB, then $L=2048$ is the threshold for leaf capacity (that is, each leaf can store a maximum of 2048 data items and a minimum of $ceil(L/2) = 1024$ data items). 

Lastly, how many leaves are in the B+ tree? Let $N$ be the total number of data items. There are between $ceil(N / L)$ and $ceil(2N / L)$ leaves in the tree.

### B+ Tree Find
Suppose we want to find 81? This search would be $\mathcal{O}(log_{m}(\#leaves))$ disk block reads, searching within each internal node would be $\mathcal{O}(log(M))$ assuming binary search, and searching within the leaf would be $\mathcal{O}(L)$ (or even better, $\mathcal{O}(log L)$ assuming binary search and the data items are kept sorted).

For example, find 81 in the following B+ tree:
<img src="https://raw.githubusercontent.com/gsprint23/cpts215/master/lessons/figures/five_way_bplustree3.png" width="800">

### B+ Tree Insert
Insertion into a B+ tree ends up maintaining all leaves at the same level before and after insertion. This could mean increasing the height of the tree.

Suppose we want to insert 57 into the following B+ tree:
<img src="https://raw.githubusercontent.com/gsprint23/cpts215/master/lessons/figures/bplustree_insert1.png" width="800">
And after inserting 57:
<img src="https://raw.githubusercontent.com/gsprint23/cpts215/master/lessons/figures/bplustree_insert2.png" width="800">
And after inserting 55:
<img src="https://raw.githubusercontent.com/gsprint23/cpts215/master/lessons/figures/bplustree_insert3.png" width="800">
And after inserting 40:
<img src="https://raw.githubusercontent.com/gsprint23/cpts215/master/lessons/figures/bplustree_insert4.png" width="800">

### B+ Tree Delete
Suppose we want to delete 99 from the following B+ tree:
<img src="https://raw.githubusercontent.com/gsprint23/cpts215/master/lessons/figures/bplustree_delete1.png" width="800">
And after deleting 99:
<img src="https://raw.githubusercontent.com/gsprint23/cpts215/master/lessons/figures/bplustree_delete2.png" width="800">

## Practice Problems
Note: the following problems are adapted from Koffman and Wolfgang.

### 1
Draw a B-tree with order 5 that stores the sequence of integers: 20, 30, 8, 10, 15, 18, 44, 26, 28, 23, 25, 43, 55, 36, 44, 39.

### 2
Show the tree in problem #1 after removing the following items one at a time: 30, 26, and 15.

### 3 
Draw a B+ tree that would be formed by inserting the integers shown in problem #1. Then, show the tree after inserting each of the following values one at a time: 1, 5, 9, 13.